1. SCRIPT DE LA BD INICIAL

    Creacion de la BD

In [1]:
IF EXISTS 
   (
     SELECT name
FROM master.dbo.sysdatabases
WHERE name = 'bichos_de_bolsillo'
    )
  DROP DATABASE bichos_de_bolsillo

GO

IF EXISTS (SELECT loginname FROM MASTER.dbo.syslogins 
       WHERE NAME = 'Administrador')
       DROP LOGIN Administrador

GO

IF EXISTS (SELECT loginname FROM MASTER.dbo.syslogins 
       WHERE NAME = 'Jugador')
       DROP LOGIN Jugador

GO

CREATE DATABASE bichos_de_bolsillo

GO

USE bichos_de_bolsillo

GO

--Tabla Tipos
CREATE TABLE Tipos
(
    tipo_id INT PRIMARY KEY IDENTITY (1 , 1),
    tipo_nombre VARCHAR(20) NOT NULL,
    resistente VARCHAR(20)
)

--Tabla Especies
CREATE TABLE Especies
(
    especie_id INT PRIMARY KEY IDENTITY (1 , 1),
    especie_daño_ataque INT NOT NULL,
    especie_velocidad_ataque INT NOT NULL,
    especie_nombre VARCHAR(20) NOT NULL,
    especie_aumento_salud INT NOT NULL,
    especie_tipo INT FOREIGN KEY REFERENCES Tipos(tipo_id)
)

--Tabla Usuarios
CREATE TABLE Usuarios
(
    usuario_id INT PRIMARY KEY IDENTITY (1,1),
    usuario_nombre VARCHAR(20) NOT NULL,
    usuario_correo VARCHAR(50) NOT NULL,
    usuario_contraseña VARCHAR(20) NOT NULL,
    usuario_tipo VARCHAR(20) NOT NULL,
    usuario_estado BIT NOT NULL DEFAULT 1
)

--Tabla Catalogo de Ataques
CREATE TABLE Catalogo_Ataques
(
    catalogo_id INT PRIMARY KEY IDENTITY (1,1),
    catalogo_nombre VARCHAR(20) NOT NULL,
    catalogo_nivel INT NOT NULL,
    catalogo_tipo INT FOREIGN KEY REFERENCES Tipos(tipo_id)
)

--Tabla Bichos
CREATE TABLE Bichos
(
    bicho_id INT PRIMARY KEY IDENTITY (1,1),
    bicho_nombre VARCHAR(20) NOT NULL,
    bicho_salud INT DEFAULT 100,
    bicho_experiencia INT DEFAULT 1,
    bicho_nivel INT DEFAULT 1,
    bicho_tipo INT FOREIGN KEY REFERENCES Tipos(tipo_id),
    bicho_especie INT FOREIGN KEY REFERENCES Especies(especie_id),
    bicho_propietario INT FOREIGN KEY REFERENCES Usuarios(usuario_id) DEFAULT NULL
)

--Tabla Recolecciones
CREATE TABLE Recolecciones
(
    recoleccion_id INT IDENTITY PRIMARY KEY,
    recoleccion_bicho INT FOREIGN KEY REFERENCES Bichos(bicho_id),
    recoleccion_fecha DATE NOT NULL,
    recoleccion_usuario INT FOREIGN KEY REFERENCES Usuarios(usuario_id)
)

--Tabla Intercambios
CREATE TABLE Intercambios
(
    intercambio_id INT PRIMARY KEY IDENTITY (1,1),
    usuario_uno INT FOREIGN KEY REFERENCES Usuarios(usuario_id),
    usuario_dos INT FOREIGN KEY REFERENCES Usuarios(usuario_id),
    bicho_uno INT FOREIGN KEY REFERENCES Bichos(bicho_id),
    bicho_dos INT FOREIGN KEY REFERENCES Bichos(bicho_id),
    intercambio_fecha DATE NOT NULL
)

--Tabla Combates
CREATE TABLE Combates
(
    combate_id INT PRIMARY KEY IDENTITY (1,1),
    usuario_uno INT FOREIGN KEY REFERENCES Usuarios(usuario_id),
    usuario_dos INT FOREIGN KEY REFERENCES Usuarios(usuario_id),
    combate_fecha DATE NOT NULL,
    ganador INT NOT NULL DEFAULT -1
)

--Tabla de Rondas
CREATE TABLE Rondas
(
    ronda_noRonda INT IDENTITY (1,1),
    combate_id INT FOREIGN KEY REFERENCES Combates(combate_id),
    bicho_uno INT FOREIGN KEY REFERENCES Bichos(bicho_id),
    bicho_dos INT FOREIGN KEY REFERENCES Bichos(bicho_id),
    ataque_uno INT FOREIGN KEY REFERENCES Catalogo_Ataques(catalogo_id),
    ataque_dos INT FOREIGN KEY REFERENCES Catalogo_Ataques(catalogo_id),
    nivel_daño_uno INT NOT NULL,
    nivel_daño_dos INT NOT NULL,
    ganador INT NOT NULL DEFAULT 1 FOREIGN KEY REFERENCES Bichos(bicho_id)

        PRIMARY KEY (ronda_noRonda, combate_id)
)

--Tabla de Ataques de Bichos
CREATE TABLE Ataques_Bicho
(
    bicho_id INT FOREIGN KEY REFERENCES Bichos(bicho_id),
    ataque_id INT FOREIGN KEY REFERENCES Catalogo_Ataques(catalogo_id),

    PRIMARY KEY(bicho_id, ataque_id)
)


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:02.923

CONSTRAINTS

In [2]:
ALTER TABLE Usuarios ADD CONSTRAINT UsuariosConstraint UNIQUE (usuario_correo)

GO

ALTER TABLE Bichos ADD CONSTRAINT BichosConstraint UNIQUE (bicho_nombre)

GO

ALTER TABLE Catalogo_Ataques ADD CONSTRAINT AtaquesConstraint UNIQUE (catalogo_nombre)

GO

ALTER TABLE Tipos ADD CONSTRAINT TiposConstraint UNIQUE (tipo_nombre)

GO

ALTER TABLE Especies ADD CONSTRAINT EspeciesConstraint UNIQUE (especie_nombre)

GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.560

3. INSERTAR DATOS

In [3]:
USE bichos_de_bolsillo

GO

--TIPOS
INSERT INTO Tipos
    (tipo_nombre, resistente)
Values
    ('Aire', 'Tierra'),
    ('Tierra', 'Aire'),
    ('Fuego', 'Agua'),
    ('Agua', 'Fuego')

--ESPECIES
INSERT INTO Especies
    (especie_nombre, especie_tipo, especie_aumento_salud, especie_daño_ataque, especie_velocidad_ataque)
VALUES
    ('Eolo', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Aire'), 20, 15, 15),
    ('Boreas', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Aire'), 30, 25, 15),
    ('Vulcano', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Fuego'), 25, 25, 20),
    ('Prometeo', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Fuego'), 15, 20, 10),
    ('Poseidon', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Agua'), 30, 30, 30),
    ('Triton', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Agua'), 20, 15, 10),
    ('Gaia', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Tierra'), 25, 30, 30),
    ('Pan', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Tierra'), 15, 20, 20)

--BICHOS
INSERT INTO Bichos
    (bicho_nombre, bicho_tipo, bicho_especie)
VALUES
    ('Charmander', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Fuego'), (SELECT especie_id
        FROM Especies
        WHERE especie_nombre = 'Vulcano')),
    ('Charmeleon', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Fuego'), (SELECT especie_id
        FROM Especies
        WHERE especie_nombre = 'Vulcano')),
    ('Ponyta', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Fuego'), (SELECT especie_id
        FROM Especies
        WHERE especie_nombre = 'Prometeo')),
    ('Rapidash', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Fuego'), (SELECT especie_id
        FROM Especies
        WHERE especie_nombre = 'Prometeo')),
    ('Vulpix', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Fuego'), (SELECT especie_id
        FROM Especies
        WHERE especie_nombre = 'Vulcano')),
    ('Squirtle', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Agua'), (SELECT especie_id
        FROM Especies
        WHERE especie_nombre = 'Poseidon')),
    ('Wartortle', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Agua'), (SELECT especie_id
        FROM Especies
        WHERE especie_nombre = 'Poseidon')),
    ('Magikarp', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Agua'), (SELECT especie_id
        FROM Especies
        WHERE especie_nombre = 'Triton')),
    ('Gyarados', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Agua'), (SELECT especie_id
        FROM Especies
        WHERE especie_nombre = 'Triton')),
    ('Psyduck', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Agua'), (SELECT especie_id
        FROM Especies
        WHERE especie_nombre = 'Poseidon')),
    ('Sandshrew', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Tierra'), (SELECT especie_id
        FROM Especies
        WHERE especie_nombre = 'Gaia')),
    ('Sandslash', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Tierra'), (SELECT especie_id
        FROM Especies
        WHERE especie_nombre = 'Gaia')),
    ('Diglett', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Tierra'), (SELECT especie_id
        FROM Especies
        WHERE especie_nombre = 'Pan')),
    ('Dugtrio ', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Tierra'), (SELECT especie_id
        FROM Especies
        WHERE especie_nombre = 'Pan')),
    ('Graveler', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Tierra'), (SELECT especie_id
        FROM Especies
        WHERE especie_nombre = 'Gaia')),
    ('Pidgey', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Aire'), (SELECT especie_id
        FROM Especies
        WHERE especie_nombre = 'Eolo')),
    ('Pidgeotto ', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Aire'), (SELECT especie_id
        FROM Especies
        WHERE especie_nombre = 'Eolo')),
    ('Spearow', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Aire'), (SELECT especie_id
        FROM Especies
        WHERE especie_nombre = 'Boreas')),
    ('Fearow', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Aire'), (SELECT especie_id
        FROM Especies
        WHERE especie_nombre = 'Boreas')),
    ('Charizard', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Aire'), (SELECT especie_id
        FROM Especies
        WHERE especie_nombre = 'Eolo'))

--CATALOGO DE ATAQUES
INSERT INTO Catalogo_Ataques
    (catalogo_nombre, catalogo_tipo, catalogo_nivel)
VALUES
    ('Puño de Fuego', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Fuego'), 1),
    ('Puño de Agua', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Agua'), 1),
    ('Puño de Tierra', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Tierra'), 1),
    ('Puño de Aire', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Aire'), 1),
    ('Huracan', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Agua'), 2),
    ('Viento', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Aire'), 2),
    ('Terremoto', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Tierra'), 2),
    ('Incendio', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Fuego'), 2),
    ('Tormenta', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Agua'), 3),
    ('Volcan', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Fuego'), 3),
    ('Arena Movediza', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Tierra'), 3),
    ('Ventisca', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Aire'), 3),
    ('Tsunami', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Agua'), 4),
    ('Rayo', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Fuego'), 4),
    ('Hierro', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Tierra'), 4),
    ('Tornado', (SELECT tipo_id
        FROM Tipos
        WHERE tipo_nombre = 'Aire'), 4)

--USUARIOS
INSERT INTO Usuarios
    (usuario_nombre, usuario_correo, usuario_contraseña, usuario_tipo)
Values
    ('Lucia', 'just.lucysm@gmail.com', '123456', 'Jugador'),
    ('Edgar', 'edgar@gmail.com', '123456', 'Jugador'),
    ('Jose', 'jose@gmail.com', '123456', 'Jugador'),
    ('Fabiola', 'fabiola@gmail.com', '123456', 'Jugador'),
    ('Diego', 'diego@gmail.com', '123456', 'Jugador'),
    ('El Patron', 'rossy@gmail.com', '12345', 'Administrador')

--RECOLECCIONES
INSERT INTO Recolecciones
    (recoleccion_fecha, recoleccion_bicho, recoleccion_usuario)
Values
    (GETDATE(), (SELECT bicho_id
        FROM Bichos
        WHERE bicho_nombre = 'Charmander'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar')),
    (GETDATE(), (SELECT bicho_id
        FROM Bichos
        WHERE bicho_nombre = 'Diglett'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose')),
    (GETDATE(), (SELECT bicho_id
        FROM Bichos
        WHERE bicho_nombre = 'Pidgey'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola')),
    (GETDATE(), (SELECT bicho_id
        FROM Bichos
        WHERE bicho_nombre = 'Pidgeotto'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego')),
    (GETDATE(), (SELECT bicho_id
        FROM Bichos
        WHERE bicho_nombre = 'Spearow'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose')),
    (GETDATE(), (SELECT bicho_id
        FROM Bichos
        WHERE bicho_nombre = 'Gyarados'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia')),
    (GETDATE(), (SELECT bicho_id
        FROM Bichos
        WHERE bicho_nombre = 'Squirtle'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego')),
    (GETDATE(), (SELECT bicho_id
        FROM Bichos
        WHERE bicho_nombre = 'Psyduck'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola')),
    (GETDATE(), (SELECT bicho_id
        FROM Bichos
        WHERE bicho_nombre = 'SandSlash'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia')),
    (GETDATE(), (SELECT bicho_id
        FROM Bichos
        WHERE bicho_nombre = 'Charmeleon'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola')),
    (GETDATE(), (SELECT bicho_id
        FROM Bichos
        WHERE bicho_nombre = 'Magikarp'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar')),
    (GETDATE(), (SELECT bicho_id
        FROM Bichos
        WHERE bicho_nombre = 'Ponyta'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego')),
    (GETDATE(), (SELECT bicho_id
        FROM Bichos
        WHERE bicho_nombre = 'Wartortle'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar')),
    (GETDATE(), (SELECT bicho_id
        FROM Bichos
        WHERE bicho_nombre = 'Graveler'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia')),
    (GETDATE(), (SELECT bicho_id
        FROM Bichos
        WHERE bicho_nombre = 'Charizard'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))

--Actualizar el valor bicho_propietario en la tabla de Bichos por las recolecciones que se hicieron
UPDATE Bichos 
SET bicho_propietario = (SELECT usuario_id
FROM Usuarios
WHERE usuario_nombre = 'Edgar')
WHERE bicho_nombre = 'Charmander'
UPDATE Bichos 
SET bicho_propietario = (SELECT usuario_id
FROM Usuarios
WHERE usuario_nombre = 'Jose')
WHERE bicho_nombre = 'Diglett'
UPDATE Bichos 
SET bicho_propietario = (SELECT usuario_id
FROM Usuarios
WHERE usuario_nombre = 'Fabiola')
WHERE bicho_nombre = 'Pidgey'
UPDATE Bichos 
SET bicho_propietario = (SELECT usuario_id
FROM Usuarios
WHERE usuario_nombre = 'Diego')
WHERE bicho_nombre = 'Pidgeotto'
UPDATE Bichos 
SET bicho_propietario = (SELECT usuario_id
FROM Usuarios
WHERE usuario_nombre = 'Jose')
WHERE bicho_nombre = 'Spearow'
UPDATE Bichos 
SET bicho_propietario = (SELECT usuario_id
FROM Usuarios
WHERE usuario_nombre = 'Lucia')
WHERE bicho_nombre = 'Gyarados'
UPDATE Bichos 
SET bicho_propietario = (SELECT usuario_id
FROM Usuarios
WHERE usuario_nombre = 'Diego')
WHERE bicho_nombre = 'Squirtle'
UPDATE Bichos 
SET bicho_propietario = (SELECT usuario_id
FROM Usuarios
WHERE usuario_nombre = 'Fabiola')
WHERE bicho_nombre = 'Psyduck'
UPDATE Bichos 
SET bicho_propietario = (SELECT usuario_id
FROM Usuarios
WHERE usuario_nombre = 'Lucia')
WHERE bicho_nombre = 'SandSlash'
UPDATE Bichos 
SET bicho_propietario = (SELECT usuario_id
FROM Usuarios
WHERE usuario_nombre = 'Fabiola')
WHERE bicho_nombre = 'Charmeleon'
UPDATE Bichos 
SET bicho_propietario = (SELECT usuario_id
FROM Usuarios
WHERE usuario_nombre = 'Edgar')
WHERE bicho_nombre = 'Magikarp'
UPDATE Bichos 
SET bicho_propietario = (SELECT usuario_id
FROM Usuarios
WHERE usuario_nombre = 'Diego')
WHERE bicho_nombre = 'Ponyta'
UPDATE Bichos 
SET bicho_propietario = (SELECT usuario_id
FROM Usuarios
WHERE usuario_nombre = 'Edgar')
WHERE bicho_nombre = 'Wartortle'
UPDATE Bichos 
SET bicho_propietario = (SELECT usuario_id
FROM Usuarios
WHERE usuario_nombre = 'Lucia')
WHERE bicho_nombre = 'Graveler'
UPDATE Bichos 
SET bicho_propietario = (SELECT usuario_id
FROM Usuarios
WHERE usuario_nombre = 'Jose')
WHERE bicho_nombre = 'Charizard'


--ATAQUES DE BICHOS
INSERT INTO Ataques_Bicho
    (bicho_id, ataque_id)
VALUES
    (1, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Fuego')),
    (1, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Huracan')),
    (1, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Terremoto')),
    (1, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Arena Movediza')),
    (1, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Rayo')),
    (2, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Tierra')),
    (2, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Viento')),
    (2, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Incendio')),
    (2, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Arena Movediza')),
    (2, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Ventisca')),
    (3, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Ventisca')),
    (3, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Volcan')),
    (3, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Hierro')),
    (3, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Tormenta')),
    (3, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Tierra')),
    (4, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Agua')),
    (4, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Viento')),
    (4, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Ventisca')),
    (4, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Aire')),
    (4, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Rayo')),
    (5, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Tierra')),
    (5, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Incendio')),
    (5, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Hierro')),
    (5, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Aire')),
    (5, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Viento')),
    (6, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Fuego')),
    (6, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Ventisca')),
    (6, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Terremoto')),
    (6, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Arena Movediza')),
    (6, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Hierro')),
    (7, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Agua')),
    (7, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Tsunami')),
    (7, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Hierro')),
    (7, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Arena Movediza')),
    (7, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Rayo')),
    (8, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Fuego')),
    (8, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Incendio')),
    (8, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Ventisca')),
    (8, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Tormenta')),
    (8, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Viento')),
    (9, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Fuego')),
    (9, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Ventisca')),
    (9, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Volcan')),
    (9, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Hierro')),
    (9, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Incendio')),
    (10, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Agua')),
    (10, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Viento')),
    (10, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Tsunami')),
    (10, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Arena Movediza')),
    (10, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Aire')),
    (11, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Ventisca')),
    (11, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Huracan')),
    (11, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Hierro')),
    (11, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Tormenta')),
    (11, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Incendio')),
    (12, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Tierra')),
    (12, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Incendio')),
    (12, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Terremoto')),
    (12, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Volcan')),
    (12, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Viento')),
    (13, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Fuego')),
    (13, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Huracan')),
    (13, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Terremoto')),
    (13, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Arena Movediza')),
    (13, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Aire')),
    (14, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Agua')),
    (14, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Huracan')),
    (14, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Hierro')),
    (14, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Arena Movediza')),
    (14, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Incendio')),
    (15, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Fuego')),
    (15, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Huracan')),
    (15, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Tsunami')),
    (15, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Tormenta')),
    (15, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Hierro')),
    (16, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Fuego')),
    (16, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Ventisca')),
    (16, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Incendio')),
    (16, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Volcan')),
    (16, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Viento')),
    (17, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Tierra')),
    (17, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Huracan')),
    (17, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Hierro')),
    (17, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Arena Movediza')),
    (17, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Rayo')),
    (18, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Fuego')),
    (18, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Aire')),
    (18, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Terremoto')),
    (18, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Tormenta')),
    (18, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Incendio')),
    (19, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Agua')),
    (19, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Ventisca')),
    (19, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Terremoto')),
    (19, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Volcan')),
    (19, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Aire')),
    (20, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Puño de Fuego')),
    (20, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Viento')),
    (20, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Tsunami')),
    (20, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Arena Movediza')),
    (20, (SELECT catalogo_id
        FROM Catalogo_Ataques
        WHERE catalogo_nombre = 'Rayo'))

--INTERCAMBIOS
INSERT INTO Intercambios
    (intercambio_fecha, usuario_uno, usuario_dos, bicho_uno, bicho_dos)
Values
    (GETDATE(), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), 1, 2),
    (GETDATE(), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), 5, 4),
    (GETDATE(), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), 6, 3),
    (GETDATE(), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), 10, 8),
    (GETDATE(), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), 12, 15),
    (GETDATE(), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), 10, 14),
    (GETDATE(), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), 9, 7),
    (GETDATE(), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), 2, 8),
    (GETDATE(), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), 4, 1),
    (GETDATE(), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), 5, 6)




Commands completed successfully.

(4 rows affected)

(8 rows affected)

(20 rows affected)

(16 rows affected)

...

(1 row affected)

(1 row affected)

(100 rows affected)

(10 rows affected)

Total execution time: 00:00:00.800

    Insertar datos en Combates y Rondas (100 c/u)

In [4]:
INSERT INTO Combates
    (usuario_uno, usuario_dos, combate_fecha)
VALUES
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'), GETDATE()),
    ((SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'), (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'), GETDATE())




INSERT INTO Rondas
    (combate_id, bicho_uno, bicho_dos, ataque_uno, ataque_dos, nivel_daño_uno, nivel_daño_dos, ganador)
VALUES
    (58, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 1),
        7, 15, 86, 22,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'))
,
    (78, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 3),
        13, 11, 24, 48,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (68, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 3),
        7, 15, 79, 66,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (1, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 3),
        12, 11, 91, 62,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (76, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 1),
        8, 7, 53, 51,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'))
,
    (96, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 3),
        2, 1, 30, 1,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (7, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 2),
        12, 2, 72, 66,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'))
,
    (10, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 1),
        5, 10, 20, 5,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'))
,
    (97, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 3),
        16, 5, 42, 70,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (88, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 3),
        16, 10, 99, 63,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (69, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 2),
        12, 11, 21, 43,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'))
,
    (27, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 1),
        9, 14, 31, 56,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (89, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 2),
        11, 11, 74, 73,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (23, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 1),
        7, 16, 84, 73,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (20, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 1),
        8, 9, 72, 13,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'))
,
    (55, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 1),
        5, 10, 9, 54,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'))
,
    (37, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 3),
        7, 14, 69, 64,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'))
,
    (33, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 1),
        10, 3, 51, 17,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'))
,
    (15, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 2),
        6, 9, 97, 96,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'))
,
    (46, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 3),
        3, 13, 9, 55,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (35, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 2),
        4, 12, 39, 23,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'))
,
    (89, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 1),
        4, 11, 59, 63,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (21, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 1),
        13, 3, 20, 84,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'))
,
    (59, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 1),
        3, 3, 42, 90,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (89, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 2),
        3, 1, 33, 63,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (3, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 3),
        8, 7, 82, 2,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'))
,
    (63, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 1),
        2, 10, 97, 6,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'))
,
    (12, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 1),
        10, 4, 2, 22,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'))
,
    (83, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 1),
        14, 5, 29, 69,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (93, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 3),
        8, 7, 51, 90,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'))
,
    (51, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 1),
        1, 6, 23, 74,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (26, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 1),
        14, 8, 78, 95,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (43, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 1),
        14, 7, 24, 48,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'))
,
    (64, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 3),
        6, 14, 52, 47,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'))
,
    (90, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 2),
        7, 6, 7, 93,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'))
,
    (68, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 1),
        15, 16, 59, 1,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (20, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 3),
        1, 11, 6, 73,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (11, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 3),
        2, 16, 70, 43,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'))
,
    (22, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 2),
        4, 10, 84, 66,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (78, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 2),
        9, 8, 29, 55,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (39, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 1),
        1, 2, 26, 29,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (97, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 2),
        8, 11, 11, 62,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (25, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 3),
        16, 6, 36, 25,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'))
,
    (38, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 3),
        6, 4, 82, 95,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (86, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 1),
        7, 16, 97, 55,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'))
,
    (72, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 3),
        12, 11, 5, 87,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (43, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 2),
        5, 1, 89, 17,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (42, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 2),
        14, 15, 75, 82,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'))
,
    (98, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 2),
        3, 3, 86, 41,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (87, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 2),
        11, 16, 14, 87,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (46, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 2),
        3, 10, 57, 69,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (5, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 2),
        15, 4, 47, 72,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (77, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 1),
        8, 14, 84, 46,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (61, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 2),
        16, 14, 0, 74,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (56, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 2),
        6, 2, 85, 98,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (73, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 3),
        5, 8, 69, 25,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'))
,
    (88, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 1),
        12, 13, 78, 30,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (80, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 1),
        6, 10, 91, 67,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'))
,
    (1, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 3),
        7, 16, 56, 56,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (75, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 1),
        15, 14, 13, 6,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (48, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 2),
        16, 14, 39, 14,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (81, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 1),
        9, 4, 49, 12,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'))
,
    (92, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 1),
        13, 3, 5, 33,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'))
,
    (97, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 2),
        13, 12, 75, 79,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (36, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 3),
        6, 9, 94, 58,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (54, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 3),
        12, 12, 78, 55,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'))
,
    (4, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 1),
        2, 14, 83, 98,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (70, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 1),
        1, 1, 12, 26,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'))
,
    (78, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 3),
        1, 6, 86, 85,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'))
,
    (2, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 3),
        6, 4, 70, 49,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'))
,
    (36, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 2),
        3, 12, 8, 54,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'))
,
    (3, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 1),
        1, 13, 33, 75,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'))
,
    (24, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 1),
        16, 4, 0, 44,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (25, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 2),
        16, 13, 68, 96,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (36, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 2),
        6, 4, 66, 78,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Diego'))
,
    (54, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 1),
        14, 11, 77, 46,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'))
,
    (85, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 1),
        9, 16, 10, 91,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'))
,
    (75, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 2),
        15, 13, 93, 68,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (31, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 3),
        11, 14, 1, 40,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (85, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 1),
        1, 12, 54, 74,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'))
,
    (42, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 1),
        4, 1, 48, 76,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'))
,
    (43, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 2),
        10, 13, 21, 7,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (19, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 1),
        8, 4, 24, 26,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'))
,
    (80, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 1),
        15, 13, 13, 47,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'))
,
    (46, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 1),
        15, 12, 25, 66,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (77, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 1),
        1, 14, 6, 70,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'))
,
    (95, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 1),
        13, 16, 95, 23,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (73, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 1),
        2, 16, 9, 87,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (50, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 3),
        5, 3, 31, 19,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'))
,
    (6, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 2),
        16, 11, 75, 85,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'))
,
    (93, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 3), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 3),
        8, 9, 53, 13,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (48, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 3),
        12, 13, 15, 19,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'))
,
    (87, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 2),
        13, 1, 14, 35,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Fabiola'))
,
    (73, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 3),
        4, 5, 28, 79,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (95, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Fabiola')) AS Bichillos
        WHERE row_number = 3),
        9, 15, 71, 58,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (83, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 3),
        16, 15, 76, 78,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (49, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 1),
        15, 7, 75, 65,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'))
,
    (6, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Edgar')) AS Bichillos
        WHERE row_number = 2),
        9, 9, 14, 59,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Edgar'))
,
    (31, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 2), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Jose')) AS Bichillos
        WHERE row_number = 2),
        4, 16, 15, 43,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Jose'))
,
    (65, (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Lucia')) AS Bichillos
        WHERE row_number = 1), (SELECT bicho_id
        FROM
            (SELECT
                ROW_NUMBER() OVER(order by (select 1)) as row_number,
                bicho_id
            FROM Bichos
            WHERE bicho_propietario = (SELECT usuario_id
            FROM Usuarios
            WHERE usuario_nombre = 'Diego')) AS Bichillos
        WHERE row_number = 3),
        2, 15, 86, 3,
        (SELECT usuario_id
        FROM Usuarios
        WHERE usuario_nombre = 'Lucia'))


(100 rows affected)

(100 rows affected)

Total execution time: 00:00:03.300

4. CONSULTA DE DATOS

In [5]:
SELECT
    tipo_id AS 'ID',
    tipo_nombre AS 'Nombre de los Tipos',
    resistente AS 'Resistente a'

FROM Tipos;


SELECT
    especie_id AS 'ID',
    especie_nombre AS 'Nombre de la Especie',
    especie_tipo AS 'ID del Tipo',
    especie_aumento_salud AS 'Aumento de Salud',
    especie_daño_ataque AS 'Daño de Ataque',
    especie_velocidad_ataque AS 'Velocidad de Ataque'

FROM Especies;


SELECT
    bicho_id AS 'ID',
    bicho_nombre AS 'Nombre del Bicho',
    bicho_salud AS 'Salud',
    bicho_tipo AS 'Tipo',
    bicho_especie AS 'Especie',
    bicho_propietario AS 'Propietario ID',
    (SELECT usuario_nombre
    FROM Usuarios
    WHERE usuario_id = bicho_propietario) AS 'Propietario Nombre'

FROM Bichos;


SELECT
    catalogo_id AS 'ID',
    catalogo_nombre AS 'Nombre del Ataque',
    catalogo_tipo AS 'ID del Tipo',
    catalogo_nivel AS 'Nivel de Ataque'

FROM Catalogo_Ataques;


SELECT
    usuario_id AS 'ID',
    usuario_nombre AS 'Nombre del Usuario'
FROM Usuarios;


SELECT
    recoleccion_fecha AS 'Fecha de Recoleccion',
    recoleccion_bicho AS 'ID Bicho',
    recoleccion_usuario AS 'Usuario'

FROM Recolecciones;

SELECT
    intercambio_id AS 'ID',
    intercambio_fecha AS 'Fecha del Intercambio',
    usuario_uno AS 'Usuario 1',
    usuario_dos AS 'Usuario 2',
    bicho_uno AS 'Bicho del Usuario1',
    bicho_dos AS 'Bicho del Usuario2'

FROM Intercambios;

SELECT
    bicho_id AS "Bicho",
    ataque_id AS "Ataque"
FROM Ataques_Bicho

SELECT
    combate_id AS 'ID',
    usuario_uno AS 'Jugador 1',
    usuario_dos AS 'Jugador 2',
    combate_fecha AS 'Fecha'
FROM Combates

SELECT
    combate_id AS 'Combate',
    bicho_uno AS 'Bicho 1',
    bicho_dos AS 'Bicho 2',
    ataque_uno AS 'Ataque 1',
    ataque_dos AS 'Ataque 2',
    nivel_daño_uno AS 'Daño 1',
    nivel_daño_dos AS 'Daño 2',
    ganador AS 'Ganador'
FROM Rondas
ORDER BY combate_id

/*11-Bichos que tiene faviola*/
select bichos.bicho_nombre as 'Bicho',tipos.tipo_nombre as 'Tipo',tipos.resistente from Bichos bichos 
inner join Tipos tipos on bicho_tipo = tipo_Id
where bicho_propietario = 4 

/*12- Bichos que tiene José con el nombre de cada bicho + su salud + que te diga a que es resistente cada bicho que posee y que te muestre su daño de ataque.*/
select bichos.bicho_nombre as 'Bicho',bichos.bicho_salud as 'Salud',tipos.resistente as 'Resistente',
catalogoAtaques.catalogo_nombre as 'Ataques'
from Bichos bichos 
inner join Tipos tipos on bicho_tipo = tipo_Id
inner join Ataques_Bicho ataquesBichos on ataquesBichos.bicho_id = bichos.bicho_id
inner join Catalogo_Ataques catalogoAtaques on catalogoAtaques.catalogo_id = ataquesBichos.ataque_id
where bicho_propietario = 3 

/*13- Bichos que existen en total y en orden alfabético.*/
select bicho_nombre as 'Bicho' from Bichos order by bicho_nombre asc ; 

/*14- Que te muestre el bicho con sus respectivos ataques con un orden de mayor a menor.*/
select bichos.bicho_nombre as 'Bicho',bichos.bicho_salud as 'Salud',tipos.resistente as 'Resistente',
catalogoAtaques.catalogo_nombre as 'Ataque',catalogoAtaques.catalogo_nivel as 'Nivel'
from Bichos bichos 
inner join Tipos tipos on bicho_tipo = tipo_Id
inner join Ataques_Bicho ataquesBichos on ataquesBichos.bicho_id = bichos.bicho_id
inner join Catalogo_Ataques catalogoAtaques on catalogoAtaques.catalogo_id = ataquesBichos.ataque_id
order by catalogoAtaques.catalogo_nivel

/*15- Que te muestre la fecha en que se llevó a cabo el combate 35,70 y 90.*/
select combate_fecha AS 'Fecha' from Combates where combate_id in (35,70,90)

/*16-muestre la fecha en que Edgar intercambio a charmander.*/
select intercambio_fecha as 'Fecha' from Intercambios
where usuario_uno = 2 and bicho_uno = 1 ;

/*17- Fecha en que los bichos de tipo fuego y de tipo aire combatieron.*/
select combate_fecha as 'Fecha' from Combates combate
inner join Rondas rondas on combate.combate_id = rondas.combate_id
inner join Bichos bicho1 on bicho1.bicho_id = rondas.bicho_uno
inner join Bichos bicho2 on bicho2.bicho_id = rondas.bicho_dos
where bicho1.bicho_tipo = 3 and bicho2.bicho_tipo = 2 

/*18- Que te muestre los ganadores de los combates del 20-30 con los bichos con los que participo el ganador.*/
select combates.combate_id as 'Combate',usuarios.usuario_nombre as 'Ganador',bichos.bicho_nombre as 'Bicho' from Combates combates 
inner join Rondas ronda on ronda.combate_id = combates.combate_id
inner join Bichos bichos on bichos.bicho_propietario = ronda.ganador
inner join Usuarios usuarios on usuario_id = ronda.ganador
where combates.combate_id between 20 and 30 order by combates.combate_id asc

/*19- Que te muestre las especies que tienen un aumento de salud de 30 y 25.*/
select (SELECT bicho_nombre
        FROM Bichos
        WHERE especie_id = bicho_id) as 'Bicho', especie_aumento_salud as 'Aumento de Salud' from Especies WHERE especie_aumento_salud in (25,30)

/*20- Muestre la fecha cuando Lucia y Edgar combatieron.*/
select combate_id as 'Combate', (SELECT usuario_nombre from Usuarios where usuario_uno = usuario_id) as 'Jugador Uno', 
(SELECT usuario_nombre from Usuarios where usuario_dos = usuario_id) as 'Jugador Dos',
combate_fecha as 'Fecha'
from Combates combates where combates.usuario_uno in (1,2) and combates.usuario_dos in (1,2)

/*21- Mostrar todas rondas por combate que gano Edgar.*/
select * from Rondas r where r.ganador = 2

/*22- Obtener los combates ganados totales de todos lo jugadores.*/
select ganador,usr.usuario_nombre as 'Usuario',count(*) as 'Rondas Ganadas' from Rondas
inner join Usuarios usr on usr.usuario_id = ganador
group by ganador,usuario_nombre

/*23- Mostrar la suma total de combates ganados de Jose.*/
select count(*) as 'Rondas Ganadas' from Rondas where ganador = 3  

/*24- Mostrar aquellos bichos que no tengan propietario.*/
select bicho_id as 'ID', bicho_nombre as 'Bicho' from Bichos b where b.bicho_propietario is null

/*25- Mostrar los bichos que sean resistente a tierra y fuego.*/
select * from Bichos bichos 
inner join Tipos tipos on bichos.bicho_tipo = tipos.tipo_id
where tipos.resistente in ('Tierra','Fuego')

/*26- Mostrar los usuarios que hayan ganado 20 o más combates.*/
select ganador as 'ID',usr.usuario_nombre as 'Usuario',count(*) as 'Rondas Ganadas' from Rondas
inner join Usuarios usr on usr.usuario_id = ganador
group by ganador,usuario_nombre having count (*) > 20

/*27- Mostrar los bichos que no han sido utilizado en combates.*/
select * from Bichos bichos where bichos.bicho_id not in 
(select bicho_uno from Rondas) or bichos.bicho_id not in (select bicho_dos from Rondas)

/*28- Mostrar el numero de veces que x bicho fue intercambiado junto con su fecha de intercambio.*/
select intercambio_fecha,(select bicho_nombre from Bichos where bicho_uno = bicho_id) as 'Bicho',
count(bicho_uno) as 'Veces cambiado' from Intercambios group by bicho_uno,intercambio_fecha

/*29- Mostrar los ataques que tiene charmander.*/
select bichos.bicho_nombre,
catalogoAtaques.catalogo_nombre
from Bichos bichos 
inner join Tipos tipos on bicho_tipo = tipo_Id
inner join Ataques_Bicho ataquesBichos on ataquesBichos.bicho_id = bichos.bicho_id
inner join Catalogo_Ataques catalogoAtaques on catalogoAtaques.catalogo_id = ataquesBichos.ataque_id
where bichos.bicho_id = 1

/*30- Mostrar la cantidad de bichos que los usuarios han intercambiado. (bicho uno)*/
select (select usuario_nombre from Usuarios where usuario_uno = usuario_id) as 'Usuario',
count(usuario_uno) as 'Numero intercambios' from Intercambios group by usuario_uno

/*31- Mostrar los ataques ordenados por nivel en forma descendente.*/
select catalogo_nombre as 'Ataque' from Catalogo_Ataques order by catalogo_nivel desc

/*32- Mostrar los bichos que se hayan intercambiado dos veces.*/
select bicho_uno,count(bicho_uno) as 'Veces cambiado' from Intercambios group by bicho_uno 
having count(bicho_uno) > 2 

/*33-Bichos que tiene Edgar*/
select bichos.bicho_nombre as 'Bicho',tipos.tipo_nombre as 'Tipo',tipos.resistente from Bichos bichos 
inner join Tipos tipos on bicho_tipo = tipo_Id
where bicho_propietario = 2 

/*34- Bichos que existen en total y en orden alfabético y orden descendente.*/
select bicho_nombre as 'Bicho' from Bichos order by bicho_nombre desc

/*35- Que te muestre la fecha en que se llevó a cabo el combate 10,20 y 30.*/
select combate_fecha AS 'Fecha' from Combates where combate_id in (10,20,30)

/*36- Mostrar los ataques que tiene charizard.*/
select bichos.bicho_nombre,
catalogoAtaques.catalogo_nombre
from Bichos bichos 
inner join Tipos tipos on bicho_tipo = tipo_Id
inner join Ataques_Bicho ataquesBichos on ataquesBichos.bicho_id = bichos.bicho_id
inner join Catalogo_Ataques catalogoAtaques on catalogoAtaques.catalogo_id = ataquesBichos.ataque_id
where bichos.bicho_id = 20

/*37-Bichos que tiene Jose*/
select bichos.bicho_nombre as 'Bicho',tipos.tipo_nombre as 'Tipo',tipos.resistente from Bichos bichos 
inner join Tipos tipos on bicho_tipo = tipo_Id
where bicho_propietario = 3

/*38- Mostrar los bichos que sean resistente a agua y aire.*/
select * from Bichos bichos 
inner join Tipos tipos on bichos.bicho_tipo = tipos.tipo_id
where tipos.resistente in ('Agua','Aire')

/*39- Fecha en que los bichos de tipo agua y de tipo tierra combatieron.*/
select combate_fecha as 'Fecha' from Combates combate
inner join Rondas rondas on combate.combate_id = rondas.combate_id
inner join Bichos bicho1 on bicho1.bicho_id = rondas.bicho_uno
inner join Bichos bicho2 on bicho2.bicho_id = rondas.bicho_dos
where bicho1.bicho_tipo = 1 and bicho2.bicho_tipo = 4 

/*40- Muestre la fecha cuando Jose y Edgar combatieron.*/
select combate_id as 'Combate', (SELECT usuario_nombre from Usuarios where usuario_uno = usuario_id) as 'Jugador Uno', 
(SELECT usuario_nombre from Usuarios where usuario_dos = usuario_id) as 'Jugador Dos',
combate_fecha as 'Fecha'
from Combates combates where combates.usuario_uno in (3,2) and combates.usuario_dos in (3,2)

/*41-Bichos que tiene Diego*/
select bichos.bicho_nombre as 'Bicho',tipos.tipo_nombre as 'Tipo',tipos.resistente from Bichos bichos 
inner join Tipos tipos on bicho_tipo = tipo_Id
where bicho_propietario = 5 

/*42- Que te muestre los ganadores de los combates del 10-20 con los bichos con los que participo el ganador.*/
select combates.combate_id as 'Combate',usuarios.usuario_nombre as 'Ganador',bichos.bicho_nombre as 'Bicho' from Combates combates 
inner join Rondas ronda on ronda.combate_id = combates.combate_id
inner join Bichos bichos on bichos.bicho_propietario = ronda.ganador
inner join Usuarios usuarios on usuario_id = ronda.ganador
where combates.combate_id between 10 and 20 order by combates.combate_id asc

/*43- Mostrar todas rondas por combate que gano Jose.*/
select * from Rondas r where r.ganador = 3

/*44-Bichos que tiene Lucia*/
select bichos.bicho_nombre as 'Bicho',tipos.tipo_nombre as 'Tipo',tipos.resistente from Bichos bichos 
inner join Tipos tipos on bicho_tipo = tipo_Id
where bicho_propietario = 1 

/*45- Mostrar los ataques que tiene vulpix.*/
select bichos.bicho_nombre,
catalogoAtaques.catalogo_nombre
from Bichos bichos 
inner join Tipos tipos on bicho_tipo = tipo_Id
inner join Ataques_Bicho ataquesBichos on ataquesBichos.bicho_id = bichos.bicho_id
inner join Catalogo_Ataques catalogoAtaques on catalogoAtaques.catalogo_id = ataquesBichos.ataque_id
where bichos.bicho_id = 5

(4 rows affected)

(8 rows affected)

(20 rows affected)

(16 rows affected)

(6 rows affected)

...

intercambio_fecha,Bicho,Veces cambiado
2020-06-03,Charmander,1
2020-06-03,Charmeleon,1
2020-06-03,Rapidash,1
2020-06-03,Vulpix,2
2020-06-03,Squirtle,1
2020-06-03,Gyarados,1
2020-06-03,Psyduck,2
2020-06-03,Sandslash,1


bicho_nombre,catalogo_nombre
Charmander,Puño de Fuego
Charmander,Huracan
Charmander,Terremoto
Charmander,Arena Movediza
Charmander,Rayo


Usuario,Numero intercambios
Lucia,2
Edgar,2
Jose,2
Fabiola,2
Diego,2


Ataque
Tsunami
Rayo
Hierro
Tornado
Tormenta
Volcan
Arena Movediza
Ventisca
Huracan
Viento


bicho_uno,Veces cambiado


**5. VISTAS**

In [6]:
-- VISTA PARA VISUALIZAR BICHOS
Create VIEW TablaBichos_V
AS
    SELECT
        bicho_id AS 'ID',
        bicho_nombre AS 'Bicho',
        bicho_salud AS 'Salud',
        (SELECT tipo_nombre
        FROM Tipos
        WHERE tipo_id = bicho_tipo) AS 'Tipo',
        (SELECT especie_nombre
        FROM Especies
        WHERE especie_id = bicho_especie) AS 'Especie',
        (SELECT usuario_nombre
        FROM Usuarios
        WHERE usuario_id = bicho_propietario) AS 'Propietario'
    FROM Bichos

GO

-- Vista para saber qué bichos no tienen propietario
CREATE VIEW BichosSinPropietario_V
AS
    SELECT 
    bicho_id AS 'ID', 
    bicho_nombre AS 'Nombre', 
    (SELECT tipo_nombre
        FROM Tipos
        WHERE tipo_id = bicho_tipo) AS 'Tipo',
        (SELECT especie_nombre
        FROM Especies
        WHERE especie_id = bicho_especie) AS 'Especie'
    FROM Bichos
    WHERE bicho_propietario IS NULL

GO

-- VISTA PARA VISUALIZAR USUARIOS
Create VIEW UsuariosResgistrados_V
as
    SELECT usuario_nombre as 'Usuarios', usuario_correo AS 'Correo', usuario_tipo AS 'Tipo'
    from Usuarios

GO

-- VISTA PARA VISUALIZAR RECOLECCIONES
CREATE VIEW Recolecciones_V
as
    SELECT (SELECT usuario_nombre
        FROM Usuarios
        WHERE usuario_id = recoleccion_usuario) as 'Usuario',
        (SELECT bicho_nombre
        FROM Bichos
        WHERE bicho_id = recoleccion_bicho) AS 'Bicho',
        recoleccion_fecha as 'Fecha de Recoleccion'
    from Recolecciones

GO

-- VISTA PARA VISUALIZAR COMBATES
CREATE VIEW Combates_V
as
    SELECT
        combate_id as 'ID',
        combate_fecha as 'Fecha',
        (SELECT usuario_nombre
        FROM Usuarios
        WHERE usuario_id = usuario_uno) as 'Jugador 1',
        (SELECT usuario_nombre
        FROM Usuarios
        WHERE usuario_id = usuario_dos) as 'Jugador 2',
        (SELECT count(combate_id)
        FROM Rondas
        WHERE combate_id = Combates.combate_id and ganador = usuario_uno) AS 'Ganadas del jugador 1',
        (SELECT count(combate_id)
        FROM Rondas
        WHERE combate_id = Combates.combate_id and ganador = usuario_dos) AS 'Ganadas del jugador 2'
    FROM Combates

GO

-- VISTA PARA VISUALIZAR BICHOS QUE TIENE UN USUARIO
CREATE VIEW UsuarioBichos_V
AS
    SELECT u.usuario_nombre AS 'Usuario', b.bicho_nombre AS 'Bicho', (SELECT especie_nombre
        FROM Especies
        WHERE especie_id = b.bicho_especie) AS 'Especie', (SELECT tipo_nombre
        FROM Tipos
        WHERE tipo_id = b.bicho_tipo) AS 'Tipo'
    FROM Usuarios u
        INNER JOIN Bichos b ON u.usuario_id = bicho_propietario

GO

-- VISTA PARA VISUALIZAR ATAQUES QUE TIENE UN BICHO
CREATE VIEW BichosAtaque_V
AS
    SELECT b.bicho_nombre AS 'Bicho', (SELECT catalogo_nombre
        FROM Catalogo_Ataques
        WHERE catalogo_id = a.ataque_id) AS 'Ataque'
    FROM
        Bichos b INNER JOIN Ataques_Bicho a ON b.bicho_id = a.bicho_id

GO

-- VISTA PARA VISUALIZAR JUGADORES
Create VIEW GetJugadores_V
as
    SELECT usuario_id AS 'ID', usuario_nombre as 'Nombre', usuario_correo AS 'Correo'
    from Usuarios
    WHERE usuario_tipo = 'Jugador'

GO

-- VISTA PARA VISUALIZAR ESPECIES
CREATE VIEW Especies_V
AS
    SELECT 
    especie_id AS 'ID', 
    especie_nombre AS 'Nombre', 
    (SELECT tipo_nombre
        FROM Tipos
        WHERE tipo_id = especie_tipo) AS 'Tipo'
    FROM Especies
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.105

    Probar vistas

In [7]:
SELECT *
from TablaBichos_V
SELECT *
FROM BichosSinPropietario_V
SELECT *
from UsuariosResgistrados_V
SELECT *
from Recolecciones_V
SELECT *
from Combates_V
SELECT *
FROM UsuarioBichos_V
SELECT *
FROM BichosAtaque_V
SELECT *
FROM GetJugadores_V

(20 rows affected)

(5 rows affected)

(6 rows affected)

(15 rows affected)

(100 rows affected)

...

Usuario,Bicho,Fecha de Recoleccion
Edgar,Charmander,2020-06-03
Jose,Diglett,2020-06-03
Fabiola,Pidgey,2020-06-03
Diego,Pidgeotto,2020-06-03
Jose,Spearow,2020-06-03
Lucia,Gyarados,2020-06-03
Diego,Squirtle,2020-06-03
Fabiola,Psyduck,2020-06-03
Lucia,Sandslash,2020-06-03
Fabiola,Charmeleon,2020-06-03


ID,Fecha,Jugador 1,Jugador 2,Ganadas del jugador 1,Ganadas del jugador 2
1,2020-06-03,Fabiola,Jose,1,1
2,2020-06-03,Edgar,Diego,1,0
3,2020-06-03,Lucia,Diego,1,1
4,2020-06-03,Diego,Fabiola,0,1
5,2020-06-03,Lucia,Jose,0,1
6,2020-06-03,Diego,Edgar,0,2
7,2020-06-03,Diego,Fabiola,1,0
8,2020-06-03,Diego,Lucia,0,0
9,2020-06-03,Edgar,Diego,0,0
10,2020-06-03,Edgar,Fabiola,1,0


Usuario,Bicho,Especie,Tipo
Edgar,Charmander,Vulcano,Fuego
Fabiola,Charmeleon,Vulcano,Fuego
Diego,Ponyta,Prometeo,Fuego
Diego,Squirtle,Poseidon,Agua
Edgar,Wartortle,Poseidon,Agua
Edgar,Magikarp,Triton,Agua
Lucia,Gyarados,Triton,Agua
Fabiola,Psyduck,Poseidon,Agua
Lucia,Sandslash,Gaia,Tierra
Jose,Diglett,Pan,Tierra


Bicho,Ataque
Charizard,Puño de Fuego
Charizard,Viento
Charizard,Arena Movediza
Charizard,Tsunami
Charizard,Rayo
Charmander,Puño de Fuego
Charmander,Huracan
Charmander,Terremoto
Charmander,Arena Movediza
Charmander,Rayo


ID,Nombre,Correo
1,Lucia,just.lucysm@gmail.com
2,Edgar,edgar@gmail.com
3,Jose,jose@gmail.com
4,Fabiola,fabiola@gmail.com
5,Diego,diego@gmail.com


**6. FUNCIONES**

In [8]:
-- FUNCIÓN PARA OBTENER LAS RONDAS DE UN COMBATE
CREATE FUNCTION dbo.GetRondas(@combate_id int)
RETURNS TABLE 
AS 
RETURN 
(
    SELECT
    ronda_noRonda AS 'No. de ronda',
    (SELECT bicho_nombre
    FROM Bichos
    WHERE bicho_id = bicho_uno) AS 'Bicho 1',
    (SELECT bicho_nombre
    FROM Bichos
    WHERE bicho_id = bicho_dos) AS 'Bicho 2',
    (SELECT catalogo_nombre
    FROM Catalogo_Ataques
    WHERE catalogo_id = ataque_uno) AS 'Ataque 1',
    (SELECT catalogo_nombre
    FROM Catalogo_Ataques
    WHERE catalogo_id = ataque_dos) AS 'Ataque 2',
    nivel_daño_uno AS 'Daño 1',
    nivel_daño_dos AS 'Daño 2',
    ganador AS 'Ganador'
FROM Rondas
WHERE combate_id = @combate_id
);

GO

-- FUNCIÓN PARA OBTENER LAS RONDAS DE UN COMBATE QUE GANÓ UN JUGADOR
CREATE FUNCTION dbo.GetRondasPerPlayer(@combate_id int, @usuario int)
RETURNS INT 
AS 
BEGIN
    RETURN (
SELECT count(combate_id)
    FROM Rondas
    WHERE combate_id = @combate_id AND ganador = @usuario
)
END;

GO

-- FUNCIÓN PARA OBTENER EL GANADOR DE UN COMBATE
CREATE FUNCTION dbo.GetGanadorCombate(@combate_id int, @usuario_uno int, @usuario_dos int)
RETURNS INT 
AS 
BEGIN
    DECLARE @ganados1 int, @ganados2 int;

    SET @ganados1 = dbo.GetRondasPerPlayer(@combate_id, @usuario_uno)

    SET @ganados2 = dbo.GetRondasPerPlayer(@combate_id, @usuario_uno)

    IF(@ganados1 > @ganados2)
        RETURN(@ganados1)
    ELSE IF(@ganados1 = @ganados2)
        RETURN(-1)
    RETURN(@ganados2)
END;

GO

-- FUNCIÓN PARA OBTENER BICHOS POR UN USUARIO EN ESPECÍFICO
CREATE FUNCTION dbo.GetBichosUsuario(@usuario_id int)
RETURNS TABLE 
AS
RETURN 
(
    SELECT
    b.bicho_id AS "ID",
    b.bicho_nombre AS 'Bicho',
    b.bicho_salud AS 'Salud',
    (SELECT tipo_nombre
    FROM Tipos
    WHERE tipo_id = b.bicho_tipo) AS 'Tipo',
    (SELECT especie_nombre
    FROM Especies
    WHERE especie_id = b.bicho_especie) AS 'Especie',
    u.usuario_nombre AS 'Propietario'
FROM
    Bichos b INNER JOIN Usuarios u ON b.bicho_propietario = u.usuario_id
WHERE b.bicho_propietario = @usuario_id
);

GO

-- FUNCIÓN PARA INICIAR SESIÓN
CREATE FUNCTION dbo.IniciarSesion(@correo VARCHAR(50), @contraseña VARCHAR(20))
RETURNS BIT
AS
BEGIN
    DECLARE @contraseñaCorrecta VARCHAR(20), @usuario_estado BIT;

    SET @contraseñaCorrecta = (SELECT usuario_contraseña
    FROM Usuarios
    WHERE usuario_correo = @correo)

    SET @usuario_estado = (SELECT usuario_estado
    FROM Usuarios
    WHERE usuario_correo = @correo)

    IF(@usuario_estado = 0)
        RETURN 0;

    IF(@contraseña = @contraseñaCorrecta)
        RETURN 1;

    RETURN 0;
END;

GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.358

Probar Funciones

In [9]:
SELECT *
FROM dbo.GetRondas(1)
SELECT dbo.GetRondasPerPlayer(1, 4) AS 'jugador 1'
SELECT dbo.GetRondasPerPlayer(1, 3) AS 'jugador 2'
SELECT dbo.GetGanadorCombate(1, 4, 3) AS 'Ganador';
SELECT *
FROM dbo.GetBichosUsuario(1)
SELECT dbo.IniciarSesion('just.lucysm@gmail.com', '123456') AS 'Iniciar Sesion Corectamente'
SELECT dbo.IniciarSesion('just.lucysm@gmail.com', '5533') AS 'Iniciar Sesion Incorrectamente'

(2 rows affected)

(1 row affected)

(1 row affected)

(1 row affected)

(3 rows affected)

...

jugador 2
1


Ganador
-1


ID,Bicho,Salud,Tipo,Especie,Propietario
9,Gyarados,100,Agua,Triton,Lucia
12,Sandslash,100,Tierra,Gaia,Lucia
15,Graveler,100,Tierra,Gaia,Lucia


Iniciar Sesion Corectamente
1


Iniciar Sesion Incorrectamente
0


**7. PROCEDIMIENTOS ALMACENADOS**

In [10]:
-- PROCEDIMIENTO PARA HACER INTERCAMBIOS SIN DEFINIR FECHA
CREATE PROCEDURE Intercambiar
    @usuario_uno int,
    @usuario_dos int,
    @bicho_uno int,
    @bicho_dos int
AS
INSERT INTO Intercambios
    (usuario_uno, usuario_dos, bicho_uno, bicho_dos, intercambio_fecha)
VALUES
    (@usuario_uno, @usuario_dos, @bicho_uno, @bicho_dos, GETDATE())

GO

-- PROCEDIMIENTO PARA CREAR RECOLECCIONES SIN DEFINIR FECHA
CREATE PROCEDURE Recolectar
    @usuario int,
    @bicho int
AS
INSERT INTO RECOLECCIONES
    (recoleccion_bicho, recoleccion_usuario, recoleccion_fecha)
VALUES
    (@usuario, @bicho, GETDATE())

GO

-- PROCEDIMIENTO PARA CREAR COMBATES SIN DEFINIR FECHA
CREATE PROCEDURE Combatir
    @usuario_uno int,
    @usuario_dos int
AS
INSERT INTO Combates
    (usuario_uno, usuario_dos, combate_fecha)
VALUES
    (@usuario_uno, @usuario_dos, GETDATE())

GO

-- PROCEDIMIENTO PARA CREAR BICHO SIN DEFINIR TIPO
CREATE PROCEDURE InsertarBicho
    @bicho_nombre VARCHAR(20),
    @bicho_especie VARCHAR(20)
AS
BEGIN
    DECLARE @especie_id INT, @tipo int;

    SET @especie_id = (SELECT especie_id
    FROM Especies
    WHERE especie_nombre = @bicho_especie)

    SET @tipo = (SELECT especie_tipo
    FROM Especies
    WHERE especie_nombre = @bicho_especie)

    INSERT INTO Bichos
        (bicho_nombre, bicho_especie, bicho_tipo)
    VALUES
        (@bicho_nombre, @especie_id, @tipo)
END

GO

-- PROCEDIMIENTO PARA REGISTRAR UN NUEVO JUGADOR SIN NECESIDAD DE ESPECIFICAR SU TIPO
CREATE PROCEDURE RegistrarJugador
    @nombre VARCHAR(20),
    @correo VARCHAR(50),
    @contraseña VARCHAR(20)
AS
BEGIN
    INSERT INTO Usuarios
        (usuario_nombre, usuario_correo, usuario_contraseña, usuario_tipo)
    VALUES
        (@nombre, @correo, @contraseña, 'Jugador')
END

GO

-- PROCEDIMIENTO PARA REGISTRAR UN NUEVO ADMINISTRADOR SIN NECESIDAD DE ESPECIFICAR SU TIPO
CREATE PROCEDURE RegistrarAdministrador
    @nombre VARCHAR(20),
    @correo VARCHAR(50),
    @contraseña VARCHAR(20)
AS
BEGIN
    INSERT INTO Usuarios
        (usuario_nombre, usuario_correo, usuario_contraseña, usuario_tipo)
    VALUES
        (@nombre, @correo, @contraseña, 'Administrador')
END

GO

-- PROCEDIMIENTO PARA DAR DE BAJA UN JUGADOR
CREATE PROCEDURE EliminarJugador
    @correo VARCHAR(50)
AS
BEGIN
    UPDATE Usuarios SET usuario_estado = 0 WHERE usuario_correo = @correo
END

GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.237

**8. TRIGGERS**

In [11]:
-- TRIGGER PARA ACTUALIZAR EL GANADOR DEL COMBATE DESPUÉS DE CADA RONDA
CREATE TRIGGER dbo.SetGanador_T ON dbo.Rondas 
AFTER INSERT 
AS 
BEGIN
    DECLARE @ganador int, @combate_id int, @usuario_uno int, @usuario_dos int

    --get combate_id from inserted
    SET @combate_id = (SELECT combate_id
    FROM inserted)

    -- get users
    SET @usuario_uno = (SELECT usuario_uno
    FROM Combates
    WHERE combate_id = @combate_id)

    SET @usuario_dos = (SELECT usuario_dos
    FROM Combates
    WHERE combate_id = @combate_id)

    -- get ganador
    SET @ganador = dbo.GetGanadorCombate(@combate_id, @usuario_uno, @usuario_dos)

    -- Set ganador in the table
    UPDATE Combates 
    SET ganador = @ganador
    WHERE combate_id = (SELECT combate_id
    FROM INSERTED)
END

GO

-- TRIGGER PARA AUMENTAR EXPERIENCIA DE UN BICHO AL GANAR DE UNA RONDA
CREATE TRIGGER dbo.AumentarExperiencia_T ON Combates 
AFTER UPDATE
AS 
BEGIN
    DECLARE @ganador int, @bicho1 int, @bicho2 int, @daño1 int, @daño2 int;

    SET @ganador = (SELECT ganador
    FROM updated)

    SET @bicho1 = (SELECT bicho_uno
    FROM updated)

    SET @bicho2 = (SELECT bicho_dos
    FROM updated)

    SET @daño1 = (SELECT nivel_daño_uno
    FROM updated)

    SET @daño2 = (SELECT nivel_daño_dos
    FROM updated)

    UPDATE Bichos SET bicho_salud = ROUND(((bicho_salud - @daño2) / 2), 0) WHERE bicho_id = @bicho1
    UPDATE Bichos SET bicho_salud = ROUND((bicho_salud - @daño1), 0) WHERE bicho_id = @bicho2

    IF(@daño1>@daño2)
        UPDATE Bichos SET bicho_experiencia = (bicho_experiencia + 1) WHERE bicho_id = @bicho1
    ELSE
        UPDATE Bichos SET bicho_experiencia = (bicho_experiencia + 1) WHERE bicho_id = @bicho2
END

GO

-- TRIGGER PARA ACTUALIZAR VALORES DE PROPIETARIOS CUANDO SE INSERTA EN INTERCAMBIOS
CREATE TRIGGER dbo.IntercambiaBicho_T ON dbo.Intercambios 
AFTER INSERT 
AS
BEGIN
    UPDATE Bichos SET bicho_propietario = (SELECT usuario_dos
    FROM inserted) WHERE bicho_id = (SELECT bicho_uno
    FROM inserted)

    UPDATE Bichos SET bicho_propietario = (SELECT usuario_uno
    FROM inserted) WHERE bicho_id = (SELECT bicho_dos
    FROM inserted)
END 

GO

-- TRIGGER PARA ACTUALIZAR VALORES DE PROPIETARIOS CUANDO SE INSERTA EN RECOLECCIONES
CREATE TRIGGER dbo.RecolectaBicho_T ON dbo.Recolecciones 
AFTER INSERT 
AS
BEGIN
    UPDATE Bichos SET bicho_propietario = (SELECT recoleccion_usuario
    FROM inserted) WHERE bicho_id = (SELECT recoleccion_bicho
    FROM inserted)
END 

GO

-- TRIGGER PARA AUMENTAR NIVEL (EVOLUCIÓN) DE UN BICHO
CREATE TRIGGER dbo.Evolucionar_T ON dbo.Combates 
AFTER UPDATE
AS 
BEGIN
    DECLARE @bicho INT, @experiencia INT;

    SET @bicho = (SELECT bicho_id
    FROM updated)

    SET @experiencia = (SELECT bicho_experiencia
    FROM updated)

    IF(@experiencia = 10)
        BEGIN
        UPDATE Bichos SET bicho_nivel = (bicho_nivel + 1) WHERE bicho_id = @bicho
        UPDATE Bichos SET bicho_experiencia = 0 WHERE bicho_id = @bicho
    END
END

GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.767

**9. Seguridad**

In [12]:
USE bichos_de_bolsillo

GO

CREATE SCHEMA Administracion

GO

ALTER SCHEMA Administracion TRANSFER dbo.Especies_V

GO

ALTER SCHEMA Administracion TRANSFER dbo.UsuariosResgistrados_V

GO

ALTER SCHEMA Administracion TRANSFER dbo.UsuarioBichos_V

GO

ALTER SCHEMA Administracion TRANSFER dbo.GetJugadores_V

GO

ALTER SCHEMA Administracion TRANSFER dbo.InsertarBicho

GO

ALTER SCHEMA Administracion TRANSFER dbo.RegistrarJugador

GO

ALTER SCHEMA Administracion TRANSFER dbo.RegistrarAdministrador

GO

ALTER SCHEMA Administracion TRANSFER dbo.EliminarJugador

GO

CREATE LOGIN Jugador WITH PASSWORD = '12345' MUST_CHANGE, DEFAULT_DATABASE=[bichos_de_bolsillo], DEFAULT_LANGUAGE=[Español], CHECK_EXPIRATION=ON, CHECK_POLICY=ON

GO

CREATE LOGIN Administrador WITH PASSWORD = '12345' MUST_CHANGE, DEFAULT_DATABASE=[bichos_de_bolsillo], DEFAULT_LANGUAGE=[Español], CHECK_EXPIRATION=ON, CHECK_POLICY=ON

GO

USE bichos_de_bolsillo

GO

CREATE USER Jugador FOR LOGIN Jugador

GO

CREATE USER Administrador FOR LOGIN Administrador

GO

CREATE ROLE Jugadores AUTHORIZATION Jugador

GO

CREATE ROLE Administradores AUTHORIZATION Administrador

GO

ALTER ROLE Jugadores ADD MEMBER Jugador

GO

ALTER ROLE Administradores ADD MEMBER Administrador

GRANT ALTER, EXECUTE, INSERT, SELECT, UPDATE ON SCHEMA :: Administracion TO Administradores

GRANT ALTER, EXECUTE, INSERT, SELECT, UPDATE ON SCHEMA :: dbo TO Administradores

GRANT ALTER, EXECUTE, INSERT, SELECT, UPDATE ON SCHEMA :: dbo TO Jugadores


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.548